In [1]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [3]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [4]:
# !pip install aif360

In [5]:
# !pip install BlackBoxAuditing==0.1.54

In [6]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [7]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [8]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


## Import dependencies

In [9]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.preprocessing import get_simple_preprocessor
from source.experiment_interface import run_exp_iter_with_disparate_impact

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [10]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels_student_performance'
DB_COLLECTION_NAME = 'exp_mult_repair_levels'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
# FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
FAIR_INTERVENTION_PARAMS_LST = [0.1]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'student_performance_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [11]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [12]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [13]:
import uuid

custom_table_fields_dct = {
    'session_uuid': str(uuid.uuid4()),
    # 'session_uuid': 'e039619e-a780-4c6c-a323-ce522182045f',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  dafe006c-f7e8-4b74-b05b-a4f5c315aa9e


## Initialize custom objects

In [14]:
import pandas as pd
from virny.datasets.base import BaseDataLoader


class StudentPerformanceDataset(BaseDataLoader):
    def __init__(self, dataset_path=None):
        df = pd.read_csv(dataset_path, delimiter=';')
        target = 'G3'
        df[target] = (df[target] >= 10) * 1

        df['G1'] = (df['G1'] >= 10) * 1
        df['G2'] = (df['G2'] >= 10) * 1

        categorical_columns = ['Mjob', 'Fjob', 'reason', 'guardian', 'sex']
        numerical_columns = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures',
                             'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health',
                             'absences', 'G1', 'G2']

        super().__init__(
            full_df=df,
            target=target,
            numerical_columns=numerical_columns,
            categorical_columns=categorical_columns,
        )

In [15]:
data_loader = StudentPerformanceDataset(os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'student-mat.csv'))
data_loader.X_data.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,Mjob,Fjob,reason,guardian,sex
0,18,4,4,2,2,0,4,3,4,1,1,3,6,0,0,at_home,teacher,course,mother,F
1,17,1,1,1,2,0,5,3,3,1,1,3,4,0,0,at_home,other,course,father,F
2,15,1,1,1,2,3,4,3,2,2,3,3,10,0,0,at_home,other,other,mother,F
3,15,4,2,1,3,0,3,2,2,1,1,5,2,1,1,health,services,home,mother,F
4,16,3,3,1,2,0,4,3,2,1,2,5,4,0,1,other,other,home,father,F


In [16]:
data_loader.X_data.shape

(395, 20)

## Run experiment iterations

### Experiment iteration 1

In [17]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [18]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [19]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   with_tuning=True,
                                   # with_tuning=False,
                                   # tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   dataset_name=metrics_computation_config.dataset_name,
                                   verbose=True)

2023-10-14 18:26:44 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[0.1]',
 'model_init_seed': 100,
 'session_uuid': 'dafe006c-f7e8-4b74-b05b-a4f5c315aa9e'}


2023-10-14 18:26:44 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([188, 365, 190, 353, 166,  75, 231, 341, 380, 267, 133, 197, 161,
            388, 110, 285, 118, 315,   7, 147],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([188, 365, 190, 353, 166,  75, 231, 341, 380, 267, 133, 197, 161,
            388, 110, 285, 118, 315,   7, 147],
           dtype='int64')


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_param:  0.1
2023/10/14, 18:26:46: Tuning RandomForestClassifier...


2023-10-14 18:26:47 experiment_interface.py INFO    : Models are tuned and saved to a file


2023/10/14, 18:26:47: Tuning for RandomForestClassifier is finished [F1 score = 1.0, Accuracy = 1.0]


Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/2 [00:00<?, ?it/s]